Go to this given URL and solve the following questions
    URL: https://www.youtube.com/@PW-Foundation/videos

    Q1. Write a python program to extract the video URL of the first five videos.
    Q2. Write a python program to extract the URL of the video thumbnails of the first five videos.
    Q3. Write a python program to extract the title of the first five videos.
    Q4. Write a python program to extract the number of views of the first five videos.
    Q5. Write a python program to extract the time of posting of video for the first five videos.
    
    Note: Save all the data scraped in the above questions in a CSV file.
    
    My CSV URL - https://github.com/arnab031/PWSkills_Assignment/blob/main/Channel.csv
    
    My deployment github link - https://github.com/arnab031/web_scrapping_assignment
    
    My deployment URL - https://webscrappingarnab.azurewebsites.net

In [38]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import logging
import re
import json

In [39]:
youtube_url = "https://www.youtube.com/@PW-Foundation/videos"

In [40]:
urlclient = urlopen(youtube_url)

In [41]:
youtube_page = urlclient.read().decode()

In [42]:
re.compile("\"videoId\":\"(\S+)\",\"thumbnail\"")

re.compile(r'"videoId":"(\S+)","thumbnail"', re.UNICODE)

In [47]:
from datetime import datetime, timedelta

def convert_date(n_days_ago):
    #print(n_days_ago)
    today = datetime.now()    
    result = (today - timedelta(days=int(n_days_ago)))
    result = result.date()
    #print(today, result)
    return result


In [49]:
re2 = re.compile("\"videoId\":\"(\S+)\",\"thumbnail\"")
selected_channel_videos_list = []
for c in re2.findall(youtube_page):
    search_text = "videoId\":\"{c}\"(.*?)watchEndpoint\":(.*?)\"videoId\":\"{c}\""
    #print(search_text.format(c=c))
    search_text = search_text.format(c=c)

    youtube_video_info = {
        "video_url": None,
        "thumbnail_url": None,
        "youtube_title": None,
        "views_count": None,
        "posting_time":None
    }

    f = re.search(search_text, youtube_page)
    if f:
        json_text2 = "{" + f.group(1)[1:-2] + "}}"
        json_dict = json.loads(json_text2)
        #print(json_dict)
        thumbnail_url = json_dict.get("thumbnail").get("thumbnails")[0]["url"]
        youtube_title = json_dict.get("title").get("runs")[0]["text"]
        views_count = json_dict.get("viewCountText").get("simpleText")
        views_count = re.sub("\D", "", views_count)
        video_url = "https://youtu.be"+json_dict.get("navigationEndpoint").get("commandMetadata").get("webCommandMetadata").get('url')
        posting_time = json_dict.get("publishedTimeText").get("simpleText")
        posting_date_time = convert_date(posting_time.split(" ")[0])
        #print(posting_date_time)
        selected_channel_videos_list.append({
            "video_url": video_url,
            "thumbnail_url": thumbnail_url,
            "youtube_title": youtube_title,
            "views_count": views_count,
            "posting_time": posting_date_time
        })
#print(selected_channel_videos_list, len(selected_channel_videos_list), sep="\n")

In [50]:
import csv
csv_columns = ['video_url','thumbnail_url','youtube_title','views_count','posting_time']

csv_file = "Channel.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in selected_channel_videos_list:
            writer.writerow(data)
except IOError:
    print("I/O error")